## Description of experiments

...

## What kinds of data are available

...

## How to access the data


Example ideas:
- Look up a broad connection type
   - print connectivity
   - plot connectivity vs distance
   - plot distribution of a few properties: amplitude, latency, decay tau   
- Pick pair of specific types (or perhaps a whole matrix row), look at type-dependent strength
   - Tlx3 -> pv/sst/vip
   - L2/3 Pyr/Pv dynamics?


In [37]:
import numpy as np
from multipatch_analysis import database as db
from multipatch_analysis.cell_class import CellClass, classify_pairs

Define two cell classes:
- presynaptic cells are L2/3 pyramidal cells
- postsynaptic cells are Pvalb positive (any layer)

In [4]:
pre_class = CellClass(target_layer='2/3', cre_type='unknown')
post_class = CellClass(cre_type='pvalb')
classes = {
    'L2/3 Sst': pre_class,
    'Pvalb': post_class,
}

Get a list of all cell pairs in the database that match these cell type criteria. We also filter here for a specific project name "mouse V1 coarse matrix" in order to exclude Other experiment types.

In [10]:
pair_query = db.pair_query(project_name="mouse V1 coarse matrix", pre_class=pre_class, post_class=post_class)
pairs = pair_query.all()
len(pairs)

374

The results of this query can also be returned as a pandas dataframe:

In [12]:
pair_df = db.dataframe(pair_query)
pair_df.head()

,id,experiment_id,pre_cell_id,post_cell_id,synapse,electrical,crosstalk_artifact,n_ex_test_spikes,n_in_test_spikes,synapse_sign,distance,meta
0,147344,4504,26407,26427,False,False,None,0,0,None,0.000216,None
1,147339,4504,26407,26423,False,False,None,0,0,None,0.000187,None
2,147318,4504,26407,26405,False,False,None,0,0,None,0.000069,None
3,147373,4504,26411,26427,False,False,None,0,0,None,0.000198,None
4,147366,4504,26411,26423,False,False,None,0,0,None,0.000165,None


Of the 374 pairs that were probed in the dataset above, only a fraction of them were found to be connected by a synapse:

In [15]:
connected_mask = pair_df['synapse'] == True
pair_df[connected_mask].head()

,id,experiment_id,pre_cell_id,post_cell_id,synapse,electrical,crosstalk_artifact,n_ex_test_spikes,n_in_test_spikes,synapse_sign,distance,meta
17,143916,4403,25787,25782,True,False,None,625,529,None,0.000173,None
109,63027,1880,11054,11052,True,False,None,260,236,None,0.000062,None
114,65576,1961,11529,11526,True,False,None,654,594,None,0.000043,None
116,64483,1930,11337,11341,True,False,None,612,492,None,0.000167,None
117,64480,1930,11337,11339,True,False,None,611,0,None,0.000067,None


In [32]:
list(pairs[0].connection_strength.__table__.columns)


[Column('id', Integer(), table=<connection_strength>, primary_key=True, nullable=False),
 Column('pair_id', Integer(), ForeignKey('pair.id'), table=<connection_strength>),
 Column('synapse_type', String(), table=<connection_strength>),
 Column('ic_n_samples', Integer(), table=<connection_strength>),
 Column('ic_crosstalk_mean', FloatType(), table=<connection_strength>),
 Column('ic_base_crosstalk_mean', FloatType(), table=<connection_strength>),
 Column('ic_amp_mean', FloatType(), table=<connection_strength>),
 Column('ic_amp_stdev', FloatType(), table=<connection_strength>),
 Column('ic_base_amp_mean', FloatType(), table=<connection_strength>),
 Column('ic_base_amp_stdev', FloatType(), table=<connection_strength>),
 Column('ic_amp_ttest', FloatType(), table=<connection_strength>),
 Column('ic_amp_ks2samp', FloatType(), table=<connection_strength>),
 Column('ic_deconv_amp_mean', FloatType(), table=<connection_strength>),
 Column('ic_deconv_amp_stdev', FloatType(), table=<connection_str

In [40]:
amps = [pair.connection_strength.ic_fit_amp for pair in pairs if pair.synapse]
amps = [amp for amp in amps if amp is not None]
print("Average PSP amplitude for %s->%s: %0.2f pA" % (pre_class, post_class, np.mean(amps)*1e12))

Average PSP amplitude for L2/3 unknown->pvalb: 535774270.88 pA


In [16]:




#########    ^      ^    ^    ##########
#########   END OF NOTEBOOK   ##########
#########                     ##########
#########                     ##########
#########   HERE BE DRAGONS   ##########
#########    v      v    v    ##########





In [6]:
%gui qt4
import pyqtgraph as pg
pg.dbg()

In [5]:
groups = classify_pairs(pairs, classes)

The synaptic physiology dataset is stored in a relational database. The most important tables are:
* `experiment` : one record per experiment performed, and links to all metadata about the conditions of each experiment
* `cell` : one record per recorded cell, with information about location, morphology, intrinsic properties, and transgenic reporters.
* `pair` : one record per _ordered pair_ of simultaneously recorded cells. From here we can find all pairs that were connected by chemical or electrical synapses, and link to the many characterized properties of these synapses. Note that _ordered pair_ means that the pair A->B is different from the pair B->A; we represent these as two separate records in the `pair` table.

[ simplified diagram of DB table relationships ]

 We can access anything in the database using SQL queries, like:

In [4]:
import pandas
from multipatch_analysis import database as db

In [10]:
session = db.get_default_session()
query = """
    select experiment.id, experiment.target_region, slice.genotype, slice.species from experiment
    join slice on slice.id=experiment.slice_id
    where project_name='mouse V1 coarse matrix'
"""
df = pandas.read_sql(query, session.bind)
df.head()

,id,target_region,genotype,species
0,3941,VisP,Vip-IRES-Cre/wt;Sst-IRES-FlpO/wt;Ai65F/wt;Ai14...,mouse
1,3958,VisP,Vip-IRES-Cre/wt;Pvalb-T2A-FlpO/wt;Ai65F/Ai65F;...,mouse
2,3968,VisP,Ntsr1-Cre_GN220/wt;Pvalb-T2A-FlpO/wt;Ai65F/wt;...,mouse
3,3974,VisP,Sst-IRES-Cre/wt;Pvalb-T2A-FlpO/Pvalb-T2A-FlpO;...,mouse
4,3984,VisP,Ntsr1-Cre_GN220/wt;Sst-IRES-FlpO/wt;Ai65F/wt;A...,mouse


However, using SQL can be a lot of work when you need to traverse the relationships between many different tables. An easier approach is to use an "object-relational mapper" that represents the same database in a simplified way. For example, getting the lists of cells 

In [ ]:
# find all connections from Sst cells onto L5 pyramidal cells
pairs = pair_query().filter()

## Scientific points to make
- Synapses are stochastic
- Synapses are dynamic
- Inhibition in neurons is not like inhibition in neural networks

## Project Ideas
- Correlations / mapping between EM and MP
- Network dynamics with mixed facilitating / depressing connections
- Can stochasticity improve learning in spiking models?